In [1]:
import os 
import sys
import numpy as np
from scipy.spatial.transform import Rotation as R 
import rospy 
import rospkg
import jupyros as jr

from std_msgs.msg import String, Header
from geometry_msgs.msg import PoseStamped, Pose, Point, Quaternion

In [2]:
# add catkin_ws context 
sys.path.append("/home/junting/franka_ws/devel/lib/python3.9/site-packages")

from src.lmp import *
from src.env.multimodal_env import MultiModalEnv
from src.config import cfg_tabletop



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Grounding Model: GLIP not available.


In [3]:
########################################
# initialize environment
########################################
rospy.init_node('jupyter_multimodal', log_level=rospy.DEBUG)
# get package root path 
pkg_root = rospkg.RosPack().get_path('instruct_to_policy')

# setup environment
cfg_tabletop['grounding_model']['model_name'] = 'embodiedgpt'
env = MultiModalEnv(cfg_tabletop)
env.reset()


[DEBUG] [1701260177.786056, 0.000000]: init_node, name[/jupyter_multimodal], pid[3619375]
[DEBUG] [1701260177.788054, 0.000000]: binding to 0.0.0.0 0
[DEBUG] [1701260177.789217, 0.000000]: bound to 0.0.0.0 34779
[DEBUG] [1701260177.790188, 0.000000]: connecting to junting-shlab 46943
[DEBUG] [1701260177.791331, 0.000000]: ... service URL is rosrpc://junting-shlab:34779
[DEBUG] [1701260177.793210, 0.000000]: [/jupyter_multimodal/get_loggers]: new Service instance
[DEBUG] [1701260177.795194, 0.000000]: ... service URL is rosrpc://junting-shlab:34779
[DEBUG] [1701260177.796230, 0.000000]: [/jupyter_multimodal/set_logger_level]: new Service instance
[INFO] [1701260178.022750, 5642.911000]: camera_left: Waiting for camera_left/color/camera_info...
[DEBUG] [1701260178.036197, 5642.918000]: connecting to junting-shlab 46943
[DEBUG] [1701260178.057079, 5642.928000]: connecting to junting-shlab 59817
[DEBUG] [1701260178.058763, 5642.929000]: connecting to junting-shlab 46943
[DEBUG] [1701260178

In [4]:
print(os.getcwd())

/home/junting/franka_ws/src/franka_fisher/instruct_to_policy/scripts


In [5]:
from src.perception.utils import load_data_from_multiview_detection
world_name = "world_6_mug_to_same_color_plate"
exclude_keywords = ["cabinet"]

current_dir = os.getcwd()
data_dir = os.path.join(current_dir, f"../data/benchmark/multiview_detection")

multiview_data = load_data_from_multiview_detection(
    data_dir=data_dir,
    world_name=world_name,
    exclude_keywords=exclude_keywords
)

# add '_0' to the end of each key in multiview_data['detections_list']
for i in range(len(multiview_data['detections_list'])):
    multiview_data['detections_list'][i] = {key + '_0': value for key, value in multiview_data['detections_list'][i].items()}

In [6]:
multiview_data['detections_list']

[{'green_plant_plate_0': [635, 513, 984, 713],
  'white_ceramic_plate_0': [1074, 484, 1364, 652],
  'red_plant_saucer_0': [975, 366, 1214, 487],
  'white_porcelain_mug_0': [1003, 433, 1131, 562]},
 {'green_plant_plate_0': [756, 360, 1005, 480],
  'white_ceramic_plate_0': [603, 480, 893, 647],
  'red_plant_saucer_0': [962, 511, 1295, 706],
  'white_porcelain_mug_0': [833, 429, 959, 558]},
 {'green_plant_plate_0': [1103, 344, 1278, 516],
  'red_plant_saucer_0': [698, 326, 864, 491]}]

In [7]:
data = env.get_sensor_data()
data.update({
    'detections_list': multiview_data['detections_list']
})

In [8]:
env.scene.update(data)

In [9]:
env.scene.visualize_3d_bboxes()
env.get_obj_name_list()

['green_plant_plate_0',
 'white_ceramic_plate_0',
 'red_plant_saucer_0',
 'white_porcelain_mug_0']

In [10]:
env.add_scene_objects_to_moveit()

[DEBUG] [1701260267.388983, 5701.268000]: connecting to junting-shlab 47529
Added object green_plant_plate_0 to planning scene
[DEBUG] [1701260267.946115, 5701.551000]: connecting to junting-shlab 47529
Added object white_ceramic_plate_0 to planning scene
[DEBUG] [1701260268.628005, 5702.012000]: connecting to junting-shlab 47529
Added object red_plant_saucer_0 to planning scene
[DEBUG] [1701260269.026889, 5702.235000]: connecting to junting-shlab 47529
Added object white_porcelain_mug_0 to planning scene


In [11]:
env.parse_adaptive_shape_grasp_pose('white_porcelain_mug_0')

[INFO] [1701260272.062021, 5704.344000]: Sending perception data to grasp detection service
[DEBUG] [1701260272.065532, 5704.346000]: connecting to junting-shlab 40139


position: 
  x: -0.2066335827112198
  y: -0.007539818994700909
  z: 1.0757216215133667
orientation: 
  x: 0.7588502149779254
  y: 0.32012445459361033
  z: 0.4477004106809462
  w: -0.3481824623314793